In [ ]:
import pandas as pd
import matplotlib as plt
from importlib import reload
import extract_features
import os


In [ ]:
json_path = 'C:\\Users\\mathiopv\\OneDrive - Charité - Universitätsmedizin Berlin\\BATTERY_LIFE\\data\\Sub005\\postop'

all_json_files = glob.glob(os.path.join(json_path, '*.json'))
filtered_files = [file for file in all_json_files if not os.path.basename(file).startswith('MetaTable') and not os.path.basename(file).startswith('Sub')]
filtered_files

In [ ]:
with open(os.path.join(json_path, 'Report_Json_Session_Report_20200605T115803.json')) as file:
            # Load the JSON data
            data = json.load(file)


In [ ]:
reload(extract_features)
SUBID = 'Sub005'
WARDCARE = 0 
extract_features.extract_MetaTable(json_path, SUBID, WARDCARE, filtered_files)


In [ ]:
# Get a list of JSON files starting with 'MetaTable'
MetaTable_json_files = [file for file in os.listdir(json_path) if file.startswith('MetaTable') and file.endswith('.json')]

# Create an empty list to store the DataFrames
dfs = []

# Iterate over the JSON files
for file in MetaTable_json_files:
    file_path = os.path.join(json_path, file)
    # Read the JSON file as a Series
    data = pd.read_json(file_path, typ='series')
    # Convert the Series to a DataFrame
    df = pd.DataFrame(data).transpose()
    # Append the DataFrame to the list
    dfs.append(df)

# Concatenate the DataFrames in the list
combined_df = pd.concat(dfs, ignore_index=True)

# Sort the combined DataFrame by the 'SessionDate' column
combined_df.sort_values('AccumulatedTherapyOnTimeSinceImplant_Initial', inplace=True)

# Print the combined and sorted DataFrame
combined_df


In [ ]:
import matplotlib.pyplot as plt

# Get the last row of the DataFrame
last_row = combined_df.iloc[-1]

# Extract the values for the variables
battery_percentage = last_row['BatteryPercentage']
overall_sensing_duration_sum = combined_df['overallSensingDurSec'].sum()/60
telemetry_duration_sum = combined_df['telemetry_durationSec'].sum()/60

# Create a figure with three subplots
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(8, 3))

# Plot the bar plot for Battery Percentage
ax1.bar('Battery Percentage', battery_percentage)

# Plot the bar plot for Overall Sensing Duration
ax2.bar('Overall Sensing Duration (Min)', overall_sensing_duration_sum)

# Plot the bar plot for Telemetry Duration
ax3.bar('Telemetry Duration (Min)', telemetry_duration_sum)
fig.tight_layout()
plt.show()


In [ ]:
dataframe_SubId = combined_df['SubID'].unique()[0]
dataframe_WardCare = combined_df['WardCare'].unique()[0]

dataframe_name = f"{dataframe_SubId}_AllMetaTable_WardCare{dataframe_WardCare}_{os.path.basename(json_path)}.json"
output_path = os.path.join(json_path, dataframe_name)
combined_df.to_json(output_path, orient='records')